# 10) Rootfinding

## Last time

* More on floating point
* Discuss Taylor Series activity
* Condition numbers

## Today

* Forward and backward error
* Computing volume of a polygon
* Rootfinding examples
* Use [Roots.jl](https://juliapackages.com/p/roots) to solve
* Introduce Bisection

In [ ]:
using Plots
default(linewidth=4)

What we're hoping for is for our problems (functions) and algorithms to be reliable (i.e., we can trust the result that our computer spits out). Reliable = well-conditioned and stable. But unfortunately

**Mathematical functions $f(x)$ can be ill-conditioned: (big $\kappa$)**
* Modeling is how we turn an abstract question into a mathematical function
* We want well-conditioned models (small $\kappa$)
* Some systems are intrinsically sensitive: fracture, chaotic systems, combustion

**Algorithms `f(x)` can be unstable**
* Unreliable, though sometimes practical
* Unstable algorithms are constructed from ill-conditioned parts

## Forward and backward error



* Activity: Read [Backward error](https://fncbook.github.io/fnc/intro/stability.html)

- Backward error measures what change to the original data would reproduce the result found by an algorithm.

![](https://fncbook.github.io/fnc/_images/backwarderror.svg)

* Let $\tilde{y} = \tilde{f}(x)$ be a computed result for the original data $x$. If there is a value $\tilde{x}$ such that

$$ f(\tilde{x})=\tilde{y} =\tilde{f}(x), $$

then we call $|\tilde{x} −x|/|x|$ the (relative) **backward error** of the result. 

* Instead of asking, "How close is to the true answer is your answer?", backward error asks, "How close to the true question is the question you answered?" 

:::{note}
Similar to the difference between Verification and Validation of your results. In Numerical Analysis, the concepts of Verification and Validation (V&V) can be oversimplified in a succinct manner by saying that "_verification_ is checking that you are solving the equations right" (verifying the numerics) and "_validation_ is checking that you are solving the right equations" (for physical or engineering problems this involves verifying the physics - often done by comparing model results with actual data given by observations). 
:::

* For ill-conditioned functions, the best we can hope for is small backward error.

* If an algorithm always produces small backward errors, then it is **stable**. But the converse is not always true: some stable algorithms may produce a large backward error.

## Exercise 10.1

* Find a function $f(x)$ that models something you're interested in
* Plot its condition number $\kappa$ as a function of $x$
* Plot the relative error (using single or double precision; compare using Julia's `big`)
* Is the relative error ever much bigger than $\kappa \epsilon_{\text{machine}}$?
* If not, can you find what caused the instability?


### Further reading: [FNC Introduction](https://fncbook.github.io/fnc/intro/overview.html)

## Stability demo: Volume of a polygon

In [ ]:
X = [1 0; 2 1; 1 3; 0 1; -1 1.5; -2 -1; .5 -2; 1 0]
R(θ) = [cos(θ) -sin(θ); sin(θ) cos(θ)]
Y = X * R(deg2rad(30))' .+ [0 0]
plot(Y[:,1], Y[:,2], seriestype=:shape, aspect_ratio=:equal, xlims=(-3, 3), ylims=(-3, 3))

In [ ]:
using LinearAlgebra
function pvolume(X)
    n = size(X, 1)
    vol = sum(det(X[i:i+1, :]) / 2 for i in 1:n-1)
end

@show ref = pvolume(Y)
[det(Y[i:i+1, :]) for i in 1:size(Y, 1)-1]

What happens if this polygon is translated, perhaps far away?

## Rootfinding

> Given $f(x)$, find $x$ such that $f(x) = 0$.

We'll work with scalars ($f$ and $x$ are just numbers) for now, but keep in mind that they can be vector-valued.

### Exercise 10.2 
Solve for $x$

* $f(x; b) = x^2 - b$
  * $x(b) = \sqrt{b}$
* $f(x; b) = \tan x - b$
  * $x(b) = \arctan b$
* $f(x) = \cos x + x - b$
  * $x(b) = ?$

We aren't given $f(x)$, but rather an algorithm `f(x)` that approximates it.
* Sometimes we get extra information, like `fp(x)` that approximates $f'(x)$
* If we have source code for `f(x)`, maybe it can be transformed "automatically" to obtain `fp(x)`

### Example: Queueing

In a [simple queueing model](https://en.wikipedia.org/wiki/Queueing_theory#Simple_two-equation_queue), there is an arrival rate and a departure (serving) rate.  While waiting in the queue, there is a probability of "dropping out". The length of the queue in this model is

$$ \text{length} = \frac{\text{arrival} - \text{drop}}{\text{departure}} $$

One model for the waiting time (where these rates are taken from exponential distributions) is

In [ ]:
wait(arrival, departure) = log(arrival / departure) / departure

plot(d -> wait(1, d), xlims=(.1, 1), xlabel="departure", ylabel="wait", label = "wait")

#### Departure rate given wait

* Easy to measure wait
* I have a limited tolerance for waiting

In [ ]:
my_wait = 0.8
plot([d -> wait(1, d) - my_wait, d -> 0], xlims=(.1, 1), xlabel="departure", ylabel="wait")

In [ ]:
import Pkg; Pkg.add("Roots")
using Roots
d0 = find_zero(d -> wait(1, d) - my_wait, 1)
plot([d -> wait(1, d) - my_wait, d -> 0], xlims=(.1, 1), xlabel="departure", ylabel="wait")
scatter!([d0], [0], marker=:circle, color=:black, title="d0 = $d0")

## Bisection Method

Bisection is a rootfinding technique that starts with an interval $[a,b]$ containing a root and does not require derivatives. Suppose $f$ is continuous. What is a **sufficient** condition for $f$ to have a root on $[a,b]$?

In [ ]:
hasroot(f, a, b) = f(a) * f(b) < 0

f(x) = cos(x) - x
plot(f, label = "cos(x) - x")

### Bisection Algorithm

In [ ]:
function bisect(f, a, b, tol)
    mid = (a + b) / 2
    if abs(b - a) < tol
        return mid
    elseif hasroot(f, a, mid)
        return bisect(f, a, mid, tol)
    else
        return bisect(f, mid, b, tol)
    end
end

x0 = bisect(f, -1, 3, 1e-5)
x0, f(x0)

### How fast does it converge?

In [ ]:
function bisect_hist(f, a, b, tol)
    mid = (a + b) / 2
    if abs(b - a) < tol
        return [mid]
    elseif hasroot(f, a, mid)
        return prepend!(bisect_hist(f, a, mid, tol), [mid])
    else
        return prepend!(bisect_hist(f, mid, b, tol), [mid])
    end
end

In [ ]:
bisect_hist(f, -1, 3, 1e-4)

### Iterative bisection: an optimized algorithm

* Data structures often optimized for appending rather than prepending.
* Saves stack space

In [ ]:
function bisect_iter(f, a, b, tol)
    hist = Float64[]
    while abs(b - a) > tol
        mid = (a + b) / 2
        push!(hist, mid)
        if hasroot(f, a, mid)
            b = mid
        else
            a = mid
        end
    end
    hist
end

In [ ]:
bisect_iter(f, -1, 3, 1e-4)

#### Let's plot the error

$$ \lvert \texttt{bisect}^k(f, a, b) - r \rvert, \quad k = 1, 2, \dotsc $$

where $r$ is the true root, $f(r) = 0$.

In [ ]:
r = bisect(f, -1, 3, 1e-14) # what are we trusting?
hist = bisect_hist(f, -1, 3, 1e-10)
scatter( abs.(hist .- r), yscale=:log10)
ks = 1:length(hist)
plot!(ks, 4 * (.5 .^ ks))

Evidently the error $e_k = x_k - x_*$ after $k$ bisections satisfies the bound
$$ |e^k| \le c 2^{-k} . $$

- The absolute error is **halved** at each step so the method converges **linearly**.

# Next time

* Discuss rootfinding examples
* Limitations of bisection
* Convergence classes
* Newton methods
  * convergence theory
  * derivation via convergence theory
  * reliability